In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
import datetime

In C:\Users\A\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\A\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\A\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\A\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\A\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.

In [2]:
data = pd.read_csv('../data/bitcoin.csv')
data.head()

date     open     high      low    close      volume  \
0  2017-09-25 9:00  4201000  4333000  4175000  4322000  132.484755   
1  2017-09-26 9:00  4317000  4418000  4311000  4321000   22.788340   
2  2017-09-27 9:00  4322000  4677000  4318000  4657000   32.269662   
3  2017-09-28 9:00  4657000  4772000  4519000  4586000   80.588243   
4  2017-09-29 9:00  4586000  4709000  4476000  4657000   59.352373   

          value  
0  5.602146e+08  
1  9.950724e+07  
2  1.448276e+08  
3  3.721860e+08  
4  2.724558e+08

In [3]:
high_prices = data['high'].values 
low_prices = data['low'].values 
mid_prices = (high_prices+low_prices)/2

In [4]:
#예측할때 사용할 일수 정하기
seq_len = 50
sequence_length = seq_len + 1

result = []
for index in range(len(mid_prices) - sequence_length):
    result.append(mid_prices[index: index + sequence_length])

In [5]:
normalized_data = []
for window in result:
    normalized_window = [((float(p)/float(window[0]))-1)for p in window]
    normalized_data.append(normalized_window)
    
result = np.array(normalized_data)

row = int(round(result.shape[0]*0.9))
train = result[:row,:]
np.random.shuffle(train)

x_train = train[:,:-1]
x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
y_train = train[:,-1]

x_test = result[row:,:-1]
x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
y_test = result[row:,-1]

x_train.shape, x_test.shape

((1166, 50, 1), (129, 50, 1))

In [6]:
model = Sequential()

model.add(LSTM(50,return_sequences=True, input_shape=(50,1)))

model.add(LSTM(64,return_sequences=False))

model.add(Dense(1,activation='linear'))

model.compile(loss='mse',optimizer='rmsprop')

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50, 50)            10400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 39,905
Trainable params: 39,905
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x_train, y_train,
         validation_data=(x_test, y_test),
         batch_size=10, #몇일씩 잘라서
         epochs=20)     #몇번 반복학습할것인가


Epoch 1/20


In [ ]:
pred = model.predict(x_test)

fig = plt.figure(facecolor='white')
ax = fig.add_subplot(111)
ax.plot(y_test, label='True')
ax.plot(pred, label='Prediction')
ax.legend()
plt.show()